In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import os
from fake_useragent import UserAgent
ua = UserAgent()

# Generate a random user-agent
user_agent = ua.random
# change working directory
os.chdir('../')

In [6]:
data = pd.read_csv(r"flint_multifields.csv")
data.columns

Index(['FID', 'Id', 'Address', 'City', 'County', 'State', 'Zip', 'Country'], dtype='object')

In [7]:
geolocator = Nominatim(user_agent=user_agent)

# Increase min_delay_seconds if needed and set a higher timeout for geocode requests
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, error_wait_seconds=10)

def full_address(row):
    return f"{row['Address']}, {row['City']}, {row['County']}, {row['State']}, {row['Zip']}, {row['Country']}"

# Assuming 'data' is your DataFrame
data['location'] = data.apply(lambda row: geocode(full_address(row), timeout=10), axis=1)
data['point'] = data['location'].apply(lambda loc: (loc.latitude, loc.longitude) if loc else None)

print(data[['Address', 'point']])

                  Address                                     point
0      911 Wickes Park Dr                    (43.39628, -83.963774)
1       812 N Harrison St  (43.423422816326536, -83.95881675510203)
2    9102 Copper Ridge Dr                                      None
3         4713 N State Rd   (43.07599524060151, -83.51819213533835)
4       4252 McCormick Dr                                      None
..                    ...                                       ...
361      6309 Reinbold Rd                                      None
362    125 W Lakeview Ave   (42.99336087755102, -83.67766304081633)
363      1146 Devenish Ln   (43.01474876694289, -83.76545120712565)
364   2159 Stonesthrow Dr    (43.08916551596486, -83.3782444663096)
365     6239 Muirfield Dr                                      None

[366 rows x 2 columns]


In [8]:
data.to_csv('flint_geocoded_revised.csv', index = False)

In [10]:
data.isnull().sum()

FID           0
Id            0
Address       0
City          0
County        0
State         0
Zip           0
Country       0
location    175
point       175
dtype: int64